In [1]:
import os
os.chdir("E://python")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
tita = pd.read_csv("titanic_data.csv")
tita.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
tita = tita.drop(columns=['Cabin','Name','Ticket'])

In [4]:
tita.isnull().sum()/tita['Survived'].count()

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Sex            0.000000
Age            0.198653
SibSp          0.000000
Parch          0.000000
Fare           0.000000
Embarked       0.002245
dtype: float64

In [5]:
tita['Age'].fillna(tita['Age'].median(skipna=True),inplace=True)
tita['Embarked'].fillna(tita['Embarked'].value_counts().idxmax(),inplace=True)

In [6]:
tita.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 62.8+ KB


In [7]:
tita_g = tita.copy()

In [8]:
tita_g = pd.get_dummies(tita_g,columns=['Pclass','Sex','Embarked'],drop_first=True)
tita_g.head(5)

,PassengerId,Survived,Age,SibSp,Parch,Fare,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,1,0,22.0,1,0,7.2500,0,1,1,0,1
1,2,1,38.0,1,0,71.2833,0,0,0,0,0
2,3,1,26.0,0,0,7.9250,0,1,0,0,1
3,4,1,35.0,1,0,53.1000,0,0,0,0,1
4,5,0,35.0,0,0,8.0500,0,1,1,0,1


In [9]:
tita_g.to_excel("E://python/titanic.xls")

In [10]:
X = tita_g.drop(['Survived'],axis=1)
y = tita_g['Survived']

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
print(X_train.shape)
print(y_test.shape)

(712, 10)
(179,)


In [12]:
logi = LogisticRegression()
logi.fit(X_train,y_train)

C:\Users\santhosh\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
log_pred = logi.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import sklearn.metrics as sm

In [15]:
confus = confusion_matrix(y_test,log_pred)
acc = accuracy_score(y_test,log_pred)
print(confus)

[[92 14]
 [24 49]]


In [16]:
print(acc)

0.7877094972067039


In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
scaler = StandardScaler()

In [19]:
tita_s = tita_g[['Age','SibSp','Parch','Fare']]

In [20]:
scaled = scaler.fit_transform(tita_s)

In [21]:
scaled_x = pd.DataFrame(scaled,columns =['Age','SibSp','Parch','Fare'])
scaled_x.head()

,Age,SibSp,Parch,Fare
0,-0.565736,0.432793,-0.473674,-0.502445
1,0.663861,0.432793,-0.473674,0.786845
2,-0.258337,-0.474545,-0.473674,-0.488854
3,0.433312,0.432793,-0.473674,0.420730
4,0.433312,-0.474545,-0.473674,-0.486337


In [22]:
tita_cat = tita_g[['Pclass_2','Pclass_3','Embarked_Q','Embarked_S','Sex_male']]
tita_scale = pd.concat([scaled_x,tita_cat],axis=1)

In [23]:
X1_train,X1_test,y1_train,y1_test = train_test_split(tita_scale,tita_g['Survived'],test_size=0.2,random_state=1)

In [24]:
from sklearn.linear_model import LogisticRegression
model_lasso = LogisticRegression(penalty='l1',solver='saga')

In [25]:
model_lasso.fit(X1_train,y1_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
model_lasso_pred = model_lasso.predict(X1_test)
sm.accuracy_score(y1_test,model_lasso_pred)

0.7932960893854749

In [27]:
model_ridge = LogisticRegression(penalty='l2')

In [28]:
model_ridge.fit(X1_train,y1_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
model_ridge_pred = model_ridge.predict(X1_test)
sm.accuracy_score(y1_test,model_ridge_pred)

0.8044692737430168

In [30]:
import warnings

In [31]:
warnings.filterwarnings('ignore')

In [32]:
from sklearn.linear_model import SGDClassifier #stochastic Gradient Descent Classifier

In [33]:
sgd = SGDClassifier(loss='log',penalty = 'elasticnet',alpha=0.01)

In [34]:
sgd.fit(X1_train,y1_train)

SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='elasticnet',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [35]:
sgd_pred = sgd.predict(X1_test)
sm.accuracy_score(y1_test,sgd_pred)

0.7988826815642458

In [36]:
sm.confusion_matrix(y1_test,sgd_pred)

array([[95, 11],
       [25, 48]], dtype=int64)

In [37]:
sm.confusion_matrix(y1_test,model_ridge_pred)

array([[93, 13],
       [22, 51]], dtype=int64)

In [38]:
sm.confusion_matrix(y1_test,model_lasso_pred)

array([[91, 15],
       [22, 51]], dtype=int64)

In [39]:
print(sm.classification_report(y1_test,sgd_pred))

              precision    recall  f1-score   support

           0       0.79      0.90      0.84       106
           1       0.81      0.66      0.73        73

    accuracy                           0.80       179
   macro avg       0.80      0.78      0.78       179
weighted avg       0.80      0.80      0.79       179



In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [41]:
#GridSearchCV
tuned_parameters = [{'C':[10**-4,10**-2,10**0,10**2,10**4]}]

In [42]:
logi_gs = GridSearchCV(LogisticRegression(),tuned_parameters)

In [43]:
logi_gs.fit(X1_train,y1_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.0001, 0.01, 1, 100, 10000]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [44]:
logi_gs_pred = logi_gs.predict(X1_test)
sm.accuracy_score(y1_test,logi_gs_pred)

0.8044692737430168

In [45]:
param_en = {'alpha':np.array([1,0.1,0.01,0.001,0.0001]),
           'l1_ratio':np.array([0.1,0.01,0.001,0.0001,1])}

In [46]:
sgd_en = GridSearchCV(SGDClassifier(loss='log',penalty = 'elasticnet',alpha=0.01),param_grid=param_en)

In [47]:
sgd_en.fit(X1_train,y1_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=SGDClassifier(alpha=0.01, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='log', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=None,
                                     penalty='elasticnet', power_t=0.5,
                                     random_state=None, shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04]),
                         'l1_ratio': array([1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e+00])},
             pre_dispatch='2*n_j

In [48]:
sgd_en.predict(X1_test)
sm.accuracy_score(y1_test,sgd_en.predict(X1_test))

0.8100558659217877

In [49]:
#RandomizedSearchCV
from scipy.stats import uniform

In [50]:
C = uniform(0.0001,1)
param_grid = dict(C=C)

In [51]:
logi_rs = RandomizedSearchCV(LogisticRegression(),param_distributions = param_grid,n_iter=100)

In [52]:
logi_rs.fit(X1_train,y1_train)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='auto', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='lbfgs', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='deprecated', n_iter=100, n_jobs=None,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000058FEC52988>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [53]:
logi_rs.best_params_

{'C': 0.09334711064456015}

In [54]:
logi_rs.predict(X1_test)
sm.accuracy_score(y1_test,logi_rs.predict(X1_test))

0.7988826815642458

In [55]:
param_grid = {'alpha':uniform(0.0001,1),'l1_ratio':uniform(0.0001,1)}

In [56]:
en_logi_rs = RandomizedSearchCV(SGDClassifier(loss='log',penalty='elasticnet'),param_distributions=param_grid,n_iter=100)

In [57]:
en_logi_rs.fit(X1_train,y1_train)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=SGDClassifier(alpha=0.0001, average=False,
                                           class_weight=None,
                                           early_stopping=False, epsilon=0.1,
                                           eta0=0.0, fit_intercept=True,
                                           l1_ratio=0.15,
                                           learning_rate='optimal', loss='log',
                                           max_iter=1000, n_iter_no_change=5,
                                           n_jobs=None, penalty='elasticnet',
                                           power_t=0.5, random_state=None,
                                           shuffle=True, tol=0.001,
                                           validatio...ose=0,
                                           warm_start=False),
                   iid='deprecated', n_iter=100, n_jobs=None,
                   param_distributions={'alpha'

In [58]:
en_logi_rs.best_params_

{'alpha': 0.008545277247715487, 'l1_ratio': 0.9659733447475699}

In [59]:
en_logi_rs.predict(X1_test)
sm.accuracy_score(y1_test,en_logi_rs.predict(X1_test))

0.7988826815642458

In [60]:
from sklearn.tree import DecisionTreeClassifier

In [61]:
dtc = DecisionTreeClassifier()

In [62]:
dtc.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [63]:
dtc.predict(X_test)
sm.accuracy_score(y_test,dtc.predict(X_test))

0.7206703910614525

In [64]:
#GridSearchCV
dtc_param = {'max_features':['auto','sqrt','log2'],'min_samples_split':[2,3,4,5,6,7,8,9,10],
             'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,11],'max_depth':[2,3,4,5,6,7,8]},

In [65]:
dtc_gs = GridSearchCV(dtc,param_grid=dtc_param)

In [66]:
dtc_gs.fit(X_train,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=({'max_depth': [2, 3, 4, 5, 6, 7, 8],
             

In [67]:
dtc_gs.predict(X_test)
sm.accuracy_score(y_test,dtc_gs.predict(X_test))

0.770949720670391

In [68]:
from scipy.stats import randint

In [69]:
dtc_param1 = {'max_features':['auto','sqrt','log2'],'max_depth':randint(2,10),
              'min_samples_split':randint(1,15),'min_samples_leaf':randint(1,15)}

In [70]:
dtc_rs = RandomizedSearchCV(dtc,param_distributions=dtc_param1,n_iter=100)

In [71]:
dtc_rs.fit(X_train,y_train)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
       

In [72]:
dtc_rs.predict(X_test)
sm.accuracy_score(y_test,dtc_rs.predict(X_test))

0.6983240223463687

In [73]:
#Model Evaluation
predict_proba = pd.DataFrame(dtc_rs.predict_proba(X_test))
predict_proba.head(5)

,0,1
0,0.067797,0.932203
1,0.890244,0.109756
2,0.067797,0.932203
3,0.650000,0.350000
4,0.067797,0.932203


In [74]:
dtc_pred = pd.DataFrame(dtc_rs.predict(X_test))

In [75]:
y_test = y_test.reset_index()

In [76]:
predications = pd.concat([y_test,dtc_pred,predict_proba],axis=1)

In [77]:
predications.columns = ['index','actual','predicted','Survived_0','Survived_1']
predications.head(10)

,index,actual,predicted,Survived_0,Survived_1
0,862,1,1,0.067797,0.932203
1,223,0,0,0.890244,0.109756
2,84,1,1,0.067797,0.932203
3,680,0,0,0.650000,0.350000
4,535,1,1,0.067797,0.932203
5,623,0,0,0.890244,0.109756
6,148,0,0,0.877551,0.122449
7,3,1,0,0.777778,0.222222
8,34,0,1,0.166667,0.833333
9,241,1,1,0.363636,0.636364


In [78]:
auc_score = sm.roc_auc_score(predications.actual,predications.Survived_1)
auc_score

0.7284181959162574

In [79]:
#ROC
#fpr,tpr,threshold = sm.roc_curve(y_test,predications.Survived_1,drop_intermediate=False)
#roc_auc = sm.auc(fpr,tpr)

#plt.plot(fpr,tpr,'b',label='ROC curve(area = %0.2f)'%auc_score)
#plt.legend(loc='lower right')
#plt.plot([0,1],[0,1],'r--')
#plt.xlim([0,1])
#plt.ylim([0,1])